# gender-classifier


Project file , description and guide
https://drive.google.com/drive/folders/1uaX90aHGTNzrDEItD0KqGXxMFPhKx1nX?usp=sharing

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("gender-classifier-DFE-791531.csv" ,encoding='latin1')

In [ ]:
df.head(4)

In [ ]:
df.shape

In [ ]:
df.info()

EDA

In [ ]:
import seaborn as sns 
sns.heatmap(df.corr(),annot = True)
plt.rcParams["figure.figsize"]=20,10

In [ ]:
df.corr().transpose()

Missing values

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df['gender:confidence'].value_counts()

In [ ]:
df = df[df['gender:confidence'] >=0.99]

In [ ]:
df['gender:confidence'].value_counts()

In [ ]:
df['gender'].value_counts()

In [ ]:
df['gender'].hist(bins=12)

Filling Missng Values

In [ ]:
df['gender'].describe()

In [ ]:
df['gender'].fillna(df['gender'].mode()[0])

In [ ]:
df.shape

In [ ]:
df['gender'].isnull().sum()

Replacing unknown values

In [ ]:
df['gender'].replace(to_replace = 'unknown' ,value = 'male' ,inplace = True)

In [ ]:
df['gender'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df['description'].describe()

In [ ]:
df['description'].fillna(df['description'].mode()[0],inplace = True)

In [ ]:
df.shape

removing nan value

In [ ]:
df.dropna()

In [ ]:
df.head(4)

In [ ]:
df.shape

In [ ]:
import re 


In [ ]:
def normalize_text(text):
    
    text = str(text)
    text = text.lower()
        
    text = re.sub('\s\W',' ',text)
    text = re.sub('\W\s',' ',text)
        
    text = re.sub('\s+',' ',text)
    text = re.sub('[^\x00-\x7F]+',' ', text)
    text = re.sub('https?:\/\/.*[\r\n]*', ' ', text)
    text = re.sub('[?!+%{}:;.,"\'()\[\]_]', '',text)
    text = re.sub('\s+',' ',text)
    return text

df['text_norm'] = [normalize_text(s) for s in df['text']]
df['description_norm'] = [normalize_text(s) for s in df['description']]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['description_norm']

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df.columns

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#vectorizer = vectorizer.fit(df.ix[train_rows, :]["text_norm"])
vectorizer.fit_transform(df['text_norm'])

In [ ]:
df.drop_duplicates(subset=None, keep='first', inplace=True)

LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
cat_col = ['_unit_state','text_norm', 'description_norm','sidebar_color','link_color', 'name', 'profileimage',
          'created', 'description', 'profile_yn','gender',]

In [ ]:
for each_col in cat_col:
    df[each_col] = le.fit_transform(df[each_col])

sklearn.neural_network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
X = df[['_golden', '_unit_state', '_trusted_judgments',
        'gender:confidence', 'profile_yn',
       'profile_yn:confidence', 'created',  'fav_number',
       'link_color', 'name', 'profileimage', 'retweet_count', 'sidebar_color',
        'tweet_count',  'tweet_id', 
       'text_norm', 'description_norm']].values
y = df['gender'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
clf.predict_proba(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
nb = MultinomialNB()
nb = nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))
accuracy_score(y_test, nb.predict(X_test))

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# prepare input data
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder()
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc
 
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc
 


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
# define the  model
model = Sequential()
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=100, batch_size=16, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))